In [106]:
import pandas as pd
import numpy as np

In [107]:
url ='http://data.covid19india.org/csv/latest/cowin_vaccine_data_districtwise.csv'
# vaccine data stored in dfvac
dfvac = pd.read_csv(url,error_bad_lines = False)

C:\Users\Alok Trivedy\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,25

In [108]:
dfvac2 = dfvac

In [109]:
# 2011 population data
dfs = pd.read_excel(r'C:\Users\Alok Trivedy\DDW_PCA0000_2011_Indiastatedist.xlsx', sheet_name='Sheet1')

In [110]:
dfpind = dfs.loc[[0],['Level','TOT_P']]

In [111]:
#For analysing district we are using modified census data where some of the district has corrected spellings and some of the districts has been added
dfc = pd.read_excel(r'C:\Users\Alok Trivedy\census.xlsx', sheet_name='Sheet1')

In [112]:
dfc1 = dfc[dfc['Level'] == 'DISTRICT']
dfc2 = dfc1[dfc1['TRU'] == 'Total']
# district wise population data
dfc3 = dfc2.loc[:,['Name','TOT_P']]

In [113]:
dfst1 = dfs[dfs['Level'] == 'STATE']
dfst2 = dfst1[dfst1['TRU'] == 'Total']
# state wise populationm data
dfstp = dfst2.loc[:,['Name','TOT_P']]

In [114]:
dfc3.head()

,Name,TOT_P
6,Kupwara,870354
9,Budgam,753745
12,Leh,133487
15,Kargil,140802
18,Punch,476835


In [115]:
# district wise vaccination data
# column number 2109 and 2110 were considered as we are taking vaccciantion data till 14 aug 2021
dfvc = dfvac.iloc[:,[3,5,2109,2110]]

In [116]:
# state wise vaccination data
# column number 2109 and 2110 were considered as we are taking vaccciantion data till 14 aug 2021
dfsc = dfvac.iloc[:,[1,2,2109,2110]]

In [117]:
dfsc.head()

,State_Code,State,14/08/2021.3,14/08/2021.4
0,NaN,NaN,First Dose Administered,Second Dose Administered
1,AN,Andaman and Nicobar Islands,22081,8110
2,AN,Andaman and Nicobar Islands,68043,29281
3,AN,Andaman and Nicobar Islands,142585,59896
4,AP,Andhra Pradesh,1364374,597346


In [118]:
dfsc= dfsc.iloc[1:,:]

In [119]:
sta = list(dfsc.State.unique())
sta.sort()

In [120]:
dfsc.rename(columns = {'14/08/2021.3':'f_dose','14/08/2021.4':'s_dose'}, inplace = True)

In [121]:
dfsc['f_dose']= pd.to_numeric(dfsc['f_dose'], errors ='ignore')
dfsc['s_dose']= pd.to_numeric(dfsc['s_dose'], errors ='ignore')

In [122]:
x = dfsc['f_dose'].sum()
y = dfsc['s_dose'].sum()
# for overall dataset all coumns of state wise were added
dfind = pd.DataFrame({'Country':'India','f_dose':x,'s_dose':y},index=[0])

In [123]:
# first dose and second dose data were divided by population of india
dfind['vaccinateddose1ratio'] = dfind['f_dose']/dfpind.iloc[0,1]
dfind['vaccinateddose2ratio'] = dfind['s_dose']/dfpind.iloc[0,1]

In [124]:
dfind = dfind.drop(['f_dose','s_dose'],axis=1)

In [125]:
dfind

,Country,vaccinateddose1ratio,vaccinateddose2ratio
0,India,0.346605,0.098849


In [126]:
s = sta[0]
temp   =  dfsc[dfsc['State'] == s]
summ1 = temp['f_dose'].sum()
summ2 = temp['s_dose'].sum()
# for state wise data , we added columns of all district belonging to a particular state
dfst = pd.DataFrame({'stateid':s,'f_dose':summ1,'s_dose':summ2},index = [0])


In [127]:
sta = sta[1:]

In [128]:
# same process is repeated for all other state
for i in sta:
    temp   =  dfsc[dfsc['State'] == i]
    summ1 = temp['f_dose'].sum()
    summ2 = temp['s_dose'].sum()
    temp2 =  pd.DataFrame({'stateid':i,'f_dose':summ1,'s_dose':summ2},index = [0])
    dfst = pd.concat([dfst,temp2],axis=0)

In [129]:
dfst = dfst.reset_index()
dfst= dfst.drop('index',axis=1)

In [130]:
dfst.head()

,stateid,f_dose,s_dose
0,Andaman and Nicobar Islands,232709,97287
1,Andhra Pradesh,18516141,6485212
2,Arunachal Pradesh,700977,199737
3,Assam,11430053,2442900
4,Bihar,25167378,4859267


In [131]:
#since all states name in population is in capital. i made all state name in capital as well
for i in dfst.stateid.unique():
    indexname = dfst[dfst['stateid']==i].index
    i = i.upper()
    dfst.loc[indexname,'stateid'] = i

In [132]:
# dfstp contains state wise population data
dfstp.head()

,Name,TOT_P
3,JAMMU & KASHMIR,12541302
72,HIMACHAL PRADESH,6864602
111,PUNJAB,27743338
174,CHANDIGARH,1055450
180,UTTARAKHAND,10086292


In [133]:
#finding common state between census and vaccination data frames
lis1 = dfstp['Name'].unique()
lis2 = dfst['stateid'].unique()
lis3 = [i for i in lis2 if i in lis1]

In [134]:
#only 30 states matched 
len(lis3)

30

In [135]:
# lis4 and lis5 conatins non matching state name from census and vaccuination data resepectively 
lis4 = list(set(lis1) - set(lis3))
lis5 = list(set(lis2) - set(lis3))

In [136]:
print(lis4)
lis5

['NCT OF DELHI', 'JAMMU & KASHMIR', 'DADRA & NAGAR HAVELI', 'DAMAN & DIU', 'ANDAMAN & NICOBAR ISLANDS']


['DELHI',
 'LADAKH',
 'JAMMU AND KASHMIR',
 'DADRA AND NAGAR HAVELI AND DAMAN AND DIU',
 'TELANGANA',
 'ANDAMAN AND NICOBAR ISLANDS']

In [137]:
# chhanging some state name in census data 
indexname = dfstp[dfstp['Name'] == 'NCT OF DELHI'].index
dfstp.loc[indexname,'Name'] = 'DELHI'
indexname = dfstp[dfstp['Name'] == 'ANDAMAN & NICOBAR ISLANDS'].index
dfstp.loc[indexname,'Name'] = 'ANDAMAN AND NICOBAR ISLANDS'
indexname = dfstp[dfstp['Name'] == 'JAMMU & KASHMIR'].index
dfstp.loc[indexname,'Name'] = 'JAMMU AND KASHMIR'

In [138]:
# daman and diu and DADRA & NAGAR HAVELI were combined 
x = dfstp[dfstp['Name'] == 'DAMAN & DIU']['TOT_P']
x = list(x)[0]
y = dfstp[dfstp['Name'] == 'DADRA & NAGAR HAVELI']['TOT_P']
y = list(y)[0]
p = pd.DataFrame({'Name' : 'DADRA AND NAGAR HAVELI AND DAMAN AND DIU','TOT_P' : (x+y)},index = [0])
dfstp = pd.concat([dfstp,p],axis=0)

In [139]:
indexname = dfstp[dfstp['Name'] == 'DAMAN & DIU'].index
dfstp = dfstp.drop(indexname)
indexname = dfstp[dfstp['Name'] == 'DADRA & NAGAR HAVELI'].index
dfstp= dfstp.drop(indexname)

In [140]:
dfstp = dfstp.reset_index()
dfstp = dfstp.drop('index',axis=1)

In [141]:
# as in census data jammu and kashmir and ladakh are still combined , we added their vaccination value in vaccination data
x = dfst[dfst['stateid'] == 'JAMMU AND KASHMIR']['f_dose']
x = list(x)[0]
a = dfst[dfst['stateid'] == 'JAMMU AND KASHMIR']['s_dose']
a= list(a)[0]
y = dfst[dfst['stateid'] == 'LADAKH']['f_dose']
y = list(y)[0]
b = dfst[dfst['stateid'] == 'LADAKH']['s_dose']
b = list(b)[0]


In [142]:
indexname = dfst[dfst['stateid'] == 'JAMMU AND KASHMIR'].index
dfst = dfst.drop(indexname)
indexname = dfst[dfst['stateid'] == 'LADAKH'].index
dfst = dfst.drop(indexname)

In [143]:
p = pd.DataFrame({'stateid' : 'JAMMU AND KASHMIR','f_dose' : (x+y),'s_dose':(a+b)},index = [0])
dfst = pd.concat([dfst,p],axis=0)

In [144]:
dfst.head()

,stateid,f_dose,s_dose
0,ANDAMAN AND NICOBAR ISLANDS,232709,97287
1,ANDHRA PRADESH,18516141,6485212
2,ARUNACHAL PRADESH,700977,199737
3,ASSAM,11430053,2442900
4,BIHAR,25167378,4859267


In [145]:
#after making modification in state data
lis1 = dfstp['Name'].unique()
lis2 = dfst['stateid'].unique()
lis3 = [i for i in lis2 if i in lis1]

In [146]:
# after making above modification , we are getting 34 states as common
len(lis3)

34

In [147]:
dfst = dfst.reset_index()
dfst = dfst.drop('index',axis=1)

In [148]:
# telangana state is removed from dataset
indexname  = dfst[dfst['stateid'] == 'TELANGANA'].index
dfst = dfst.drop(indexname)


In [149]:
# for all states population data for a particular state from census data are appended to vacciantion data frame for ratio calculation
for i in dfst.stateid.unique():
    indexname  = dfstp[dfstp['Name'] == i].index
    indexname2 = dfst[dfst['stateid'] == i].index 
    x = dfstp.loc[indexname,'TOT_P'].values
    x = x[0]
    dfst.loc[indexname2 , 'population'] = x

In [150]:
dfst['f_dose']= pd.to_numeric(dfst['f_dose'], errors ='ignore')
dfst['s_dose']= pd.to_numeric(dfst['s_dose'], errors ='ignore')
dfst['population']= pd.to_numeric(dfst['population'], errors ='ignore')

In [151]:
dfst['vaccinateddose1ratio'] = dfst['f_dose']/dfst['population']
dfst['vaccinateddose2ratio'] = dfst['s_dose']/dfst['population']

In [152]:
dfst = dfst.drop(['f_dose','s_dose','population'],axis=1)

In [153]:
dfst.head()

,stateid,vaccinateddose1ratio,vaccinateddose2ratio
0,ANDAMAN AND NICOBAR ISLANDS,0.611457,0.255628
1,ANDHRA PRADESH,0.218917,0.076675
2,ARUNACHAL PRADESH,0.506586,0.144347
3,ASSAM,0.366282,0.078284
4,BIHAR,0.241763,0.046679


In [154]:
# district wise vaccination data
dfvc.head()

,District_Key,District,14/08/2021.3,14/08/2021.4
0,NaN,NaN,First Dose Administered,Second Dose Administered
1,AN_Nicobars,Nicobars,22081,8110
2,AN_North and Middle Andaman,North and Middle Andaman,68043,29281
3,AN_South Andaman,South Andaman,142585,59896
4,AP_Anantapur,Anantapur,1364374,597346


In [155]:
# district wise population data
dfc3.head()

,Name,TOT_P
6,Kupwara,870354
9,Budgam,753745
12,Leh,133487
15,Kargil,140802
18,Punch,476835


In [165]:
# common district between census and vaccination data
lic1 = list(dfc3['Name'])
lic2 = list(dfvc['District'])
lic3 = [i for i in lic1 if i in lic2]

intially we were getting only 595 district as common , then we found data that some of the district in census data  ,space is added in the string , we removed those spaces , also there were some spelling difference in two district name , we used diff.xlsc(prepared manually) to remove spelling difference. and we were able to get 622 district as common, then we added 13 district manually .

In [166]:
dfc3.head()

,Name,TOT_P
6,Kupwara,870354
9,Budgam,753745
12,Leh,133487
15,Kargil,140802
18,Punch,476835


In [167]:
## removing extra space from the strings
for i in range(len(lic1)):
    l = len(lic1[i])
    s = lic1[i]
    if(ord(s[l-1]) == 32):
        print(s+' '+str(len(s)))
        indexname = dfc3[dfc3['Name'] == s].index
        s = lic1[i][:l-1]
        dfc3.loc[indexname,'Name'] = s
        

In [168]:
lic1 = list(dfc3['Name'])
lic2 = list(dfvc['District'])
lic3 = [i for i in lic1 if i in lic2]

In [169]:
# in modified district data we are getting 634 district common
len(lic3)

621

In [170]:
lic4 = list(set(lic1) - set(lic3))
lic4

['Mahbubnagar',
 'Lahul and Spiti',
 'Pashchimi Singhbhum',
 'Banas Kantha',
 'Panch Mahal',
 'Dadra & Nagar Haveli',
 'Sabar Kantha',
 'Rangareddy',
 'Mumbai Suburban',
 'Bangaluru Rural',
 'Belgavi',
 'Barddhaman',
 'Gulbarga',
 'Nicobar',
 'Bangaluru Urban',
 'Sri Potti Sriramulu Nellore',
 'Mewat',
 'Tumkur',
 'Bara Banki']

In [175]:
# some district were added manually
indexname = dfc3[dfc3['Name'] == 'Banas Kantha'].index
s = 'Banaskantha'
print(indexname)
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Mahbubnagar'].index
s = 'Mahabubnagar'
#print(indexname)
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Lahul and Spiti'].index
s = 'Lahaul and Spiti'
#print(indexname)
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Panch Mahal'].index
s = 'Panchmahal'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Dadra & Nagar Haveli'].index
s = 'Dadra and Nagar Haveli'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Sabar Kantha'].index
s = 'Sabarkantha'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Rangareddy'].index
s = 'Ranga Reddy'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Bangaluru Rural'].index
s = 'Bengaluru Rural'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Belgavi'].index
s = 'Belagavi'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Nicobar'].index
s = 'Nicobars'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Bangaluru Urban'].index
s = 'Bengaluru Urban'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Sri Potti Sriramulu Nellore'].index
s = 'S.P.S. Nellore'
dfc3.loc[indexname,'Name'] = s
indexname = dfc3[dfc3['Name'] == 'Bara Banki'].index
s = 'Barabank'
dfc3.loc[indexname,'Name'] = s


Int64Index([], dtype='int64')


In [172]:
lic5 = list(set(lic2) - set(lic3))

In [173]:
lic5 = lic5[1:]

In [174]:
lic5.sort()
lic5

['Agar Malwa',
 'Alipurduar',
 'Amethi',
 'Aravalli',
 'Balod',
 'Baloda Bazar',
 'Bametara',
 'Banaskantha',
 'Barabanki',
 'Belagavi',
 'Bengaluru Rural',
 'Bengaluru Urban',
 'Bhadradri Kothagudem',
 'Biswanath',
 'Botad',
 'Charaideo',
 'Charkhi Dadri',
 'Chengalpattu',
 'Chhota Udaipur',
 'Dadra and Nagar Haveli',
 'Devbhumi Dwarka',
 'East Jaintia Hills',
 'Fazilka',
 'Gariaband',
 'Gaurela Pendra Marwahi',
 'Gir Somnath',
 'Gomati',
 'Hapur',
 'Hojai',
 'Jagtial',
 'Jangaon',
 'Jayashankar Bhupalapally',
 'Jhargram',
 'Jiribam',
 'Jogulamba Gadwal',
 'Kakching',
 'Kalaburagi',
 'Kalimpong',
 'Kallakurichi',
 'Kamareddy',
 'Kamjong',
 'Kamle',
 'Kangpokpi',
 'Khowai',
 'Komaram Bheem',
 'Kondagaon',
 'Kra Daadi',
 'Lahaul and Spiti',
 'Lepa Rada',
 'Longding',
 'Lower Siang',
 'Mahabubabad',
 'Mahabubnagar',
 'Mahisagar',
 'Majuli',
 'Mancherial',
 'Medchal Malkajgiri',
 'Morbi',
 'Mulugu',
 'Mungeli',
 'Nagarkurnool',
 'Namsai',
 'Narayanpet',
 'Nicobars',
 'Nirmal',
 'Noney',
 

In [69]:
len(lic5)

123

In [176]:
lic1 = list(dfc3['Name'])
lic2 = list(dfvc['District'])
lic3 = [i for i in lic1 if i in lic2]

In [177]:
len(lic3)

633

In [178]:
# importing diff.xlsc file for spelling modification ( manually prepared)
diff =  pd.read_excel (r'C:\Users\Alok Trivedy\dm assign\difference district.xlsx', sheet_name='Sheet1')
diff= diff.drop('Unnamed: 2',axis=1)
arr1 = list(diff['neighbor json'])
arr2 = list(diff['districtwise'])

In [179]:
for i in range(len(arr1)):
    arr = arr1[i].rsplit(' ')
    s = ''
    if(len(arr)==1):
        s = arr[0].capitalize()
    else:
        for k in range(len(arr)-1):
            s = s+ arr[k].capitalize() + ' '
            s = s+arr[len(arr)-1].capitalize()
    arr1[i] = s
for i in range(len(arr2)):
    arr = arr2[i].rsplit(' ')
    s = ''
    if(len(arr)==1):
        s = arr[0].capitalize()
    else:
        for k in range(len(arr)-1):
            s = s+ arr[k].capitalize() + ' '
            s = s+arr[len(arr)-1].capitalize()
    arr2[i] = s

In [180]:
for i in range(len(arr1)):
    if(ord(arr1[i][0]) == 32):
        s = arr1[i][1:]
        arr1[i] = s
for i in range(len(arr2)):
    if(ord(arr2[i][0]) == 32):
        s = arr2[i][1:]
        arr2[i] = s

In [181]:
for i in range(len(lic4)):
    for j in range(len(arr1)):
        if(lic4[i] == arr1[j]):
            print(lic4[i])
            indexname = dfc3[dfc3['Name'] == lic4[i]].index
            dfc3.loc[indexname,'Name'] = arr2[j]

Rangareddy
Tumkur


In [182]:
lic1 = list(dfc3['Name'])
lic2 = list(dfvc['District'])
lic3 = [i for i in lic1 if i in lic2]

In [183]:
lic4 = list(set(lic1) - set(lic3))
lic4

['Pashchimi Singhbhum',
 'Mumbai Suburban',
 'Barabank',
 'Barddhaman',
 'Mewat',
 'Gulbarga']

In [184]:
df1 = dfc3
for i in lic4:
    indexname = df1[df1['Name'] == i ].index
    df1 =  df1.drop(indexname )
    

In [185]:
len(df1.Name.unique())

628

In [186]:
dfvc1 = dfvc
for j in lic5:
    indexname = dfvc1[dfvc1['District'] == j ].index
    dfvc1 =  dfvc1.drop(indexname )

In [187]:
dfvc1 = dfvc1.loc[1:,:]

In [188]:
dfvc1.head()

,District_Key,District,14/08/2021.3,14/08/2021.4
2,AN_North and Middle Andaman,North and Middle Andaman,68043,29281
3,AN_South Andaman,South Andaman,142585,59896
4,AP_Anantapur,Anantapur,1364374,597346
5,AP_Chittoor,Chittoor,1565591,640864
6,AP_East Godavari,East Godavari,1927490,552473


In [189]:
df1.head()

,Name,TOT_P
6,Kupwara,870354
9,Budgam,753745
12,Leh,133487
15,Kargil,140802
18,Punch,476835


In [190]:
dfvc1.rename(columns = {'14/08/2021.3':'f_dose','14/08/2021.4':'s_dose'}, inplace = True)

In [191]:
dfvc1['vaccinateddose1ratio'] = 0
dfvc1['vaccinateddose2ratio'] = 0
dfvc1['population'] = 0

In [192]:
dfvc1.head()

,District_Key,District,f_dose,s_dose,vaccinateddose1ratio,vaccinateddose2ratio,population
2,AN_North and Middle Andaman,North and Middle Andaman,68043,29281,0,0,0
3,AN_South Andaman,South Andaman,142585,59896,0,0,0
4,AP_Anantapur,Anantapur,1364374,597346,0,0,0
5,AP_Chittoor,Chittoor,1565591,640864,0,0,0
6,AP_East Godavari,East Godavari,1927490,552473,0,0,0


In [193]:
for i in dfvc1.District.unique():
    indexname  = df1[df1['Name'] == i].index
    indexname2 = dfvc1[dfvc1['District'] == i].index 
    x = df1.loc[indexname,'TOT_P'].values
    x = x[0]
    dfvc1.loc[indexname2 , 'population'] = x

In [194]:
dfvc1['f_dose']= pd.to_numeric(dfvc1['f_dose'], errors ='ignore')
dfvc1['s_dose']= pd.to_numeric(dfvc1['s_dose'], errors ='ignore')
dfvc1['population']= pd.to_numeric(dfvc1['population'], errors ='ignore')

In [195]:
dfvc1['vaccinateddose1ratio'] = dfvc1['f_dose']/dfvc1['population']
dfvc1['vaccinateddose2ratio'] = dfvc1['s_dose']/dfvc1['population']

In [196]:
dfvc1.head()

,District_Key,District,f_dose,s_dose,vaccinateddose1ratio,vaccinateddose2ratio,population
2,AN_North and Middle Andaman,North and Middle Andaman,68043,29281,0.644365,0.277290,105597
3,AN_South Andaman,South Andaman,142585,59896,0.598739,0.251514,238142
4,AP_Anantapur,Anantapur,1364374,597346,0.334311,0.146367,4081148
5,AP_Chittoor,Chittoor,1565591,640864,0.375076,0.153535,4174064
6,AP_East Godavari,East Godavari,1927490,552473,0.373958,0.107187,5154296


In [197]:
dfvc1 = dfvc1.loc[:,['District_Key','vaccinateddose1ratio','vaccinateddose2ratio']]

In [198]:
dfvc1.sort_values('vaccinateddose1ratio',inplace=True)

In [199]:
dfvc1.to_csv('district-vaccinated-dose-ratio.csv', index=False)

In [200]:
dfind.to_csv('overall-vaccination-dose-ratio.csv', index=False)

In [201]:
dfst.sort_values('vaccinateddose1ratio',inplace=True)

In [202]:
dfst.to_csv('state-vaccination-dose-ratio.csv', index=False)